## Obtener palabras
modelo.show_topic(topic_id, topn=num_words)
## Obtener documentos
modelo.get_document_topics(doc)

In [1]:
import pandas as pd
import pickle
import bz2
from IPython.display import display, HTML
from gensim.corpora import Dictionary, MmCorpus

## Seleccionar el mejor resultado 

In [2]:
with open('recursos/resultados_modelos.pkl', 'rb') as archivo:
    resultados = pickle.load(archivo)
# Para Bigramas
# with open('recursos/resultados_modelos_bi.pkl', 'rb') as archivo:
#     resultados = pickle.load(archivo)

In [7]:
resultados["lda"]['resultados_ejecuciones']
#Para bigramas
#resultados["lda-bi"]['resultados_ejecuciones']

[{'adjusted_rand_score': 0.16911014422085652,
  'normalized_mutual_info_score': 0.3039122328854189,
  'confusion_matrix': array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [5, 6, 9, ..., 0, 0, 0],
         ...,
         [0, 1, 1, ..., 0, 0, 0],
         [0, 0, 1, ..., 0, 0, 0],
         [0, 1, 0, ..., 0, 0, 0]], dtype=int64),
  'fowlkes_mallows_score': 0.24012121098352485,
  'bcubed_precision': 0.3497683956460666,
  'bcubed_recall': 0.17158452253994497,
  'bcubed_f1': 0.19114170693726434},
 {'adjusted_rand_score': 0.18742663586506847,
  'normalized_mutual_info_score': 0.31593765619920516,
  'confusion_matrix': array([[ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 1,  5, 62, ...,  0,  0,  0],
         ...,
         [ 0,  5,  9, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  6, ...,  0,  0,  0]], dtype=int64),
  'fowlkes_mallows_score': 0.2548049384456862,
  'bcubed_precision': 0.35243559408734904,


In [11]:
lista_resultados = resultados["lda"]['resultados_ejecuciones']
# Para bigramas
# lista_resultados = resultados["lda-bi"]['resultados_ejecuciones']
# Crear un DataFrame de pandas
df = pd.DataFrame(lista_resultados)

# Eliminar la columna 'confusion_matrix' ya que no se puede representar directamente en HTML
df = df.drop(columns=['confusion_matrix'])

# Convertir el DataFrame a HTML
tabla_html = df.to_html(index=False)

# Imprimir la tabla HTML
display(HTML(tabla_html))

adjusted_rand_score,normalized_mutual_info_score,fowlkes_mallows_score,bcubed_precision,bcubed_recall,bcubed_f1
0.169110,0.303912,0.240121,0.349768,0.171585,0.191142
0.187427,0.315938,0.254805,0.352436,0.193398,0.211177
0.198267,0.314867,0.268527,0.363448,0.189468,0.209311
0.149365,0.296686,0.218607,0.338920,0.166766,0.180294
0.196805,0.317096,0.267108,0.360440,0.179940,0.203617


In [18]:
# Calcular la media para cada fila
medias = df[['adjusted_rand_score', 'normalized_mutual_info_score', 'bcubed_f1']].mean(axis=1)

# Añadir las medias como una nueva columna al DataFrame
df['Media'] = medias

df_ordenado = df.sort_values(by='Media',ascending=False)

# Crear un nuevo dataframe solo con las medias
df_medias = df_ordenado[['Media']]

# Convertir el DataFrame a HTML
tabla_html_medias = df_medias.to_html()

display(HTML(tabla_html_medias))

,Media
2,0.240815
4,0.239173
1,0.238180
0,0.221388
3,0.208781


## Obtener las palabras más representativas de cada tópico 

In [3]:
etiquetas_predichas=resultados["lda"]['etiquetas_predichas'][2]
#etiquetas_predichas=resultados["lda-bi"]['etiquetas_predichas'][4]

In [4]:
with open('recursos/modelos_topicos/lda_ejecucion_567.pkl', 'rb') as archivo:
    modelo = pickle.load(archivo)
    
# with open('recursos/modelos_topicos_bi/lda-bi_ejecucion_1001.pkl', 'rb') as archivo:
#     modelo = pickle.load(archivo)

In [5]:
from collections import defaultdict

# Contar la cantidad de documentos para cada tópico
topicos_documentos = defaultdict(int)
for etiqueta in etiquetas_predichas:
    topicos_documentos[etiqueta] += 1

# Ordenar los tópicos por la cantidad de documentos (de mayor a menor)
topicos_ordenados = sorted(topicos_documentos.items(), key=lambda x: x[1], reverse=True)

# Obtener los 20 tópicos con más documentos
top_20_topicos = topicos_ordenados[:20]

# Imprimir los resultados
print("Los 20 tópicos con más documentos:")
for topico_id, cantidad_documentos in top_20_topicos:
    print(f"  Tópico {topico_id}: {cantidad_documentos} documentos")

Los 20 tópicos con más documentos:
  Tópico 25: 4446 documentos
  Tópico 54: 4235 documentos
  Tópico 28: 3987 documentos
  Tópico 39: 3496 documentos
  Tópico 41: 3480 documentos
  Tópico 6: 2830 documentos
  Tópico 47: 2658 documentos
  Tópico 35: 2499 documentos
  Tópico 34: 2259 documentos
  Tópico 42: 2019 documentos
  Tópico 49: 2006 documentos
  Tópico 59: 1993 documentos
  Tópico 8: 1842 documentos
  Tópico 2: 1673 documentos
  Tópico 58: 1593 documentos
  Tópico 23: 1543 documentos
  Tópico 29: 1540 documentos
  Tópico 57: 1521 documentos
  Tópico 9: 1441 documentos
  Tópico 51: 1341 documentos


In [6]:
topicos_ordenados_ids = [topico_id for topico_id, cantidad_documentos in top_20_topicos]

In [41]:
def generar_palabras_topicos_excel(modelo_lda, lista_topicos, num_words=15, output_excel_path="palabras_topicos_lda.xlsx"):
    # Crear una lista para almacenar los datos de la tabla
    data = []

    # Obtener las palabras más probables para cada tópico en la lista
    for topic_id in lista_topicos:
        palabras_probables = modelo_lda.show_topic(topic_id, topn=num_words)
        # Extraer solo las palabras y unirlas en una cadena separada por comas
        palabras = [palabra for palabra, probabilidad in palabras_probables]
        palabras_str = ", ".join(palabras)
        # Agregar la cadena a la lista de datos
        data.append({"Tópico": topic_id, "Palabras": palabras_str})

    # Crear un DataFrame de pandas a partir de los datos
    df = pd.DataFrame(data)

    # Exportar el DataFrame a un archivo Excel
    df.to_excel(output_excel_path, index=False)

    print(f"Las palabras de los tópicos se han exportado a '{output_excel_path}'.")



In [42]:
generar_palabras_topicos_excel(modelo, topicos_ordenados_ids)

Las palabras de los tópicos se han exportado a 'palabras_topicos_lda.xlsx'.


### Generar los documentos más relevantes de un tópico 

In [12]:
import pandas as pd

def generar_documentos_topicos_excel(modelo_lda, corpus, documentos, lista_topicos, output_excel_path="documentos_topicos_lda.xlsx"):
    """
    Genera un archivo Excel donde cada fila representa un documento, y las columnas representan las probabilidades
    de que dicho documento pertenezca a cada uno de los tópicos de la lista proporcionada.
    Solo se incluyen los 5 documentos más relevantes por cada tópico.
    
    Args:
        modelo_lda: El modelo LDA entrenado.
        corpus: El corpus de documentos procesados.
        documentos: Lista de los documentos originales.
        lista_topicos: Lista de los IDs de los tópicos de interés.
        output_excel_path: Ruta del archivo Excel de salida.
    """
    
    # Crear una lista para almacenar los datos de la tabla
    data = []

    # Recorrer cada tópico de la lista de tópicos
    for topic_id in lista_topicos:
        documentos_topicos = []

        # Recorrer cada documento
        for doc_id, doc in enumerate(corpus):
            # Obtener la distribución de tópicos para el documento
            distribucion_topicos = modelo_lda.get_document_topics(doc)
            
            # Buscar la probabilidad del tópico deseado
            probabilidad_topico = 0
            for topico, probabilidad in distribucion_topicos:
                if topico == topic_id:
                    probabilidad_topico = probabilidad
                    break
            
            # Agregar el documento y su probabilidad del tópico
            documentos_topicos.append((doc_id, probabilidad_topico))

        # Ordenar los documentos por la probabilidad del tópico (de mayor a menor)
        documentos_ordenados = sorted(documentos_topicos, key=lambda x: x[1], reverse=True)

        # Obtener los 5 documentos más relevantes para el tópico
        top_5_documentos = documentos_ordenados[:5]

        # Agregar los datos al archivo Excel
        for doc_id, probabilidad in top_5_documentos:
            contenido_documento = documentos[doc_id][:300]  # Tomamos los primeros 300 caracteres del documento
            data.append({"Documento": contenido_documento, f"Tópico_{topic_id}": probabilidad})

    # Crear un DataFrame de pandas a partir de los datos
    df = pd.DataFrame(data)

    # Exportar el DataFrame a un archivo Excel
    df.to_excel(output_excel_path, index=False)

    print(f"Los documentos más relevantes por tópico se han exportado a '{output_excel_path}'.")


In [7]:
#df_preprocesado = pd.read_csv('df_preprocesado.csv', sep=';')

In [13]:
#documentos=df_preprocesado['texto']
with bz2.BZ2File('recursos/documentos_originales.pkl.bz2', 'rb') as f:
    documentos = pickle.load(f)
corpus = MmCorpus('recursos/corpus.mm')
generar_documentos_topicos_excel(modelo,corpus,documentos,topicos_ordenados_ids)

Los documentos más relevantes por tópico se han exportado a 'documentos_topicos_lda.xlsx'.
